# Lab2 - PuLP Library

<b>Information on group members:</b><br>
1) 156035, Kuba Czech <br>
2) 156045, Wojciech Nagórka

In [1]:
from pulp import *
import numpy as np
import pandas as pd

## Introduction - brief tutorial on PuLP

In [2]:
# Create an LpProblem instance; LpMaximize = objective function is to be maximized
model = LpProblem(name="some-problem", sense=LpMaximize)

In [3]:
# Initialize the decision variables. We can set the name and lower bound as well.
# To create an array of variables, you can use comprehensions of LpProblem.dicts.

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

In [4]:
# An example expression
expression = 3 * x1 + 2 * x2
type(expression)

pulp.pulp.LpAffineExpression

In [5]:
# An example constraint
constraint = 2 * x1 + 3 * x2 >= 5
type(constraint)

pulp.pulp.LpConstraint

Ok, let's now use PuLP to solve the below problem:
$max$ $4x_1 + 2x_2$ <br>
s.t.<br>
     $1x_1 + 1x_2 \geq 1$ <br>
     $2x_1 + 1x_2 \leq 6$ <br>
     $-1x_1 + x_2 = 1$ <br>
     $x_1 \geq 0$ <br>
     $x_2 \geq 0$ 

In [6]:
# Problem
model = LpProblem(name="test-problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

model += (1 * x1 + 1*x2 >= 1, "#1 constraint")
model += (2 * x1 + 1*x2 <= 6, "#2 constraint")
model += (-1 * x1 + 1*x2 == 1, "#3 constraint")

# Objective function
obj_func = 4*x1 + 2 * x2
model += obj_func

model

test-problem:
MAXIMIZE
4*x1 + 2*x2 + 0
SUBJECT TO
#1_constraint: x1 + x2 >= 1

#2_constraint: 2 x1 + x2 <= 6

#3_constraint: - x1 + x2 = 1

VARIABLES
x1 Continuous
x2 Continuous

In [7]:
# Solve the problem
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kuba/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/5fec02e263484496b8ebb99b3e379e5b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/5fec02e263484496b8ebb99b3e379e5b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 12
After Postsolve, objective 12, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from 

In [8]:
# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [9]:
# Print objective value
print(f"objective: {model.objective.value()}")

objective: 12.000000199999999


In [10]:
# Print constraint values
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

#1_constraint: 3.3333334
#2_constraint: 9.999999983634211e-08
#3_constraint: 0.0


In [11]:
model.variables()

[x1, x2]

In [12]:
print(x1.value())
print(x2.value())

1.6666667
2.6666667


### The same code but using dictionaries and other nice tricks


In [13]:
model = LpProblem(name="some-problem", sense=LpMaximize)

In [14]:
var_names = ["First", "Second"]
x = LpVariable.dicts("x", var_names, 0)
x

{'First': x_First, 'Second': x_Second}

In [15]:
const_names = ["GE", "LE", "EQ"]
sense = [1, -1, 0] # GE, LE, EQ
coefs = [[1,1],[2,1],[-1,1]] # Matrix coefs
rhs = [1, 6, 1] 

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    print([c[i] for i in range(2)])
    print(c, s, r, cn)
    expr = lpSum([x[var_names[i]] * c[i] for i in range(2)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
    
obj_coefs = [4,2]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(2)])
    
model

[1, 1]
[1, 1] 1 1 GE
[2, 1]
[2, 1] -1 6 LE
[-1, 1]
[-1, 1] 0 1 EQ


some-problem:
MAXIMIZE
4*x_First + 2*x_Second + 0
SUBJECT TO
GE: x_First + x_Second >= 1

LE: 2 x_First + x_Second <= 6

EQ: - x_First + x_Second = 1

VARIABLES
x_First Continuous
x_Second Continuous

In [16]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names: print(x[n].value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kuba/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/7428dd12a1db431980d7148ef4a85ec7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/7428dd12a1db431980d7148ef4a85ec7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 17 RHS
At line 21 BOUNDS
At line 22 ENDATA
Problem MODEL has 3 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-6) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 12
After Postsolve, objective 12, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 12 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from 

# Homework - use the PuLP library to solve the following OR problem 

Johnson & Sons company manufactures 6 types of toys. Each toy is produced by utilizing at least one Machine 1-4, requiring a different production time (see Table below). For instance, product A requires 4 minutes on Machine 1, 4 minutes on Machine 2, 0 Minutes on Machine 3, and 0 minutes on Machine 4 (all machines must be utilized to produce a toy unless the production time equals 0). Each machine is available for a different number of hours per week. The company aims to identify the number (product-mix) of toys that must be manufactured to maximize the income (can be continuous). Notably, each toy can be sold for a different price. Due to market expectations, the company wants to manufacture twice as many F toys as A. Furthermore, the number of toys B should equal C. Solve this problem using the PuLP library. Prepare a report (in the jupyter notebook) containing information on the following:
<ol>
<li>The number of toys to manufacture;</li>
<li>The expected income;</li>
<li>The production time required on each machine;</li>
</ol>
Consider the total and partial values, i.e., considered for each toy A-F separately (e.g., income resulting from selling toy A). Also, answer the following questions concerning the found solution:
<ol>
<li>Which toy(s) production should be focused on?  </li>
<li>Is there a toy that can be excluded from consideration for production? </li>
<li>Is there a machine that is not fully utilized?</li>
</ol>

| Toy | Machine 1 | Machine 2 | Machine 3 | Machine 4 | Selling price |
| --- | --- | --- | --- | --- | --- |
| A | 4 (minutes!) | 4 | 0 | 0 | 2.50 USD |
| B | 0 | 3 | 3 | 0 | 1.00 USD |
| C | 5 | 0 | 2 | 5 | 4.00 USD |
| D | 2 | 4 | 0 | 4 | 3.00 USD |
| E | 0 | 4 | 5 | 2 | 3.50 USD |
| F | 2 | 2 | 1 | 1 | 4.00 USD |
| Production time limit (hours!) | 120 | 60 |  80 |  120 |  |

In [ ]:
A = [4, 4, 0, 0]
B = [0, 3, 3, 0]
C = [5, 0, 2, 5]
D = [2, 4, 0, 4]
E = [0, 4, 5, 2]
F = [2, 2, 1, 1]
rhs = [120 * 60, 60 * 60, 80 * 60, 120*60]
sense = [-1 for _ in range(4)]
const_names = ['Machine1', 'Machine2', 'Machine3', 'Machine4']

my_table = np.array([A, B, C, D, E, F])
my_table = my_table.T
my_table

array([[4, 0, 5, 2, 0, 2],
       [4, 3, 0, 4, 4, 2],
       [0, 3, 2, 0, 5, 1],
       [0, 0, 5, 4, 2, 1]])

In [ ]:
model = LpProblem(name="toy-optimization", sense=LpMaximize)

var_names = ['A', 'B', 'C', 'D', 'E', 'F']
x = LpVariable.dicts("x", var_names, 0)#, cat = 'Integer')

for c, s, r, cn in zip(my_table, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(6)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)

model += LpConstraint(e = lpSum([x[var_names[5]] * 1, x[var_names[0]] * (-2)]), sense = 0, rhs = 0, name = 'constraint 5')
model += LpConstraint(e = lpSum([x[var_names[1]] * 1, x[var_names[2]] * (-1)]), sense = 0, rhs = 0, name = 'constraint 6')

    
obj_coefs = [2.5, 1.0, 4.0, 3.0, 3.5, 4.0]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(6)])

status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
solution = []
for n in var_names: 
    print(x[n].value())
    solution.append(x[n].value())
solution

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Kuba/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/b5d50e2f3a5a48c3b62593f7f2dcadc7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/nx_b_wh17dg77b9kb95gxng80000gp/T/b5d50e2f3a5a48c3b62593f7f2dcadc7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 39 RHS
At line 46 BOUNDS
At line 47 ENDATA
Problem MODEL has 6 rows, 6 columns and 21 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-2) rows, 4 (-2) columns and 14 (-7) elements
0  Obj -0 Dual inf 16.749996 (4)
2  Obj 5700
Optimal - objective value 5700
After Postsolve, objective 5700, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 5700 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed 

[105.88235, 917.64706, 917.64706, 0.0, 0.0, 211.76471]

In [36]:
solution = np.array(solution)
machine_util = my_table @ solution
diff = rhs - machine_util
print([round(i, 2) for i in diff])

[1764.71, 0.0, -0.0, 2400.0]


# **Report** #

### **1. How many toys will be manufactured?** ###

In [37]:
print(f"There will be manufactured {sum(solution)} toys in total")

There will be manufactured 2152.94118 toys in total


### **2. Expected Income** ###

In [38]:
print(f"Expected income will be {solution @ obj_coefs} USD")

Expected income will be 5700.000015 USD


### **3. Production Time Required on Each Machine** ###

In [39]:
machine_util = my_table @ solution
for i in range(len(machine_util)):
    print(f"{const_names[i]} will be working for {machine_util[i]} minutes, i. e. {round(machine_util[i]/60, 2)} hours")

Machine1 will be working for 5435.29412 minutes, i. e. 90.59 hours
Machine2 will be working for 3599.9999999999995 minutes, i. e. 60.0 hours
Machine3 will be working for 4800.000010000001 minutes, i. e. 80.0 hours
Machine4 will be working for 4800.000010000001 minutes, i. e. 80.0 hours


### **4. Which toy should the production be focused on?** ###

This depends on measure we take into account.

1. If we take into account which toy has the highest income per one minute of work

In [40]:
for i in range(len(var_names)):
    print(f"For toy {var_names[i]} income per minute is equal to: {round(obj_coefs[i]/sum(my_table.T[i]), 3)}")

For toy A income per minute is equal to: 0.312
For toy B income per minute is equal to: 0.167
For toy C income per minute is equal to: 0.333
For toy D income per minute is equal to: 0.3
For toy E income per minute is equal to: 0.318
For toy F income per minute is equal to: 0.667


Then production should be most focused on toy F (and thus also toy A - there must be twice as much F toys as A toys) 

2. If we take into account which toy has the highest total income, then production should be most focused on toy C and thus B

In [41]:
for i in range(len(solution)):
    print(f"Total income for toy {var_names[i]} is equal to {solution[i] * obj_coefs[i]} USD")

Total income for toy A is equal to 264.705875 USD
Total income for toy B is equal to 917.64706 USD
Total income for toy C is equal to 3670.58824 USD
Total income for toy D is equal to 0.0 USD
Total income for toy E is equal to 0.0 USD
Total income for toy F is equal to 847.05884 USD


Then production must be focused on toy C (and rhus also toy B - there must be as many toys C as toys B)

### **5. Is there a toy that can be excluded from production?** ###

Yes, there can be excluded toys D and E from production since they were not produced at all and income was 0.00 USD

### **6. Is there any machine that is not fully utilized?** ###

In [46]:
machine_util = my_table @ solution
diff = rhs - machine_util
print([abs(round(i, 2)) for i in diff])

[1764.71, 0.0, 0.0, 2400.0]


Yes, Machine A and Machine D are not fully utilized